In [3]:
# Amend to set wd to STB_social_media_analytics.
%cd /home/zyf0717/git_environment/STB_social_media_analytics/

/home/zyf0717/git_environment/STB_social_media_analytics


In [4]:
import pandas as pd
import mysql.connector
import re
import yaml
import utils
from selenium import webdriver
from datetime import datetime
from time import sleep

In [7]:
with open('config_file.yml') as file:
    configs = yaml.load(file, Loader=yaml.FullLoader)

cnx = mysql.connector.connect(host=configs['host'],
                              database=configs['database'],
                              user=configs['user'],
                              password=configs['password'])

driver = webdriver.Chrome('./chromedriver')

# Primary key is 'POI_index'.
TripA_POI_cols = ['POI_index',
                  'total_reviews',
                  'ranking',
                  'rating_1_proportion',
                  'rating_2_proportion',
                  'rating_3_proportion',
                  'rating_4_proportion',
                  'rating_5_proportion',
                  'about',
                  'price',
                  'address',
                  'latitude',
                  'longitude',
                  'attributes_crawled_time'
                 ]

# Primary key is 'review_index', autoincrement in DB.
TripA_reviews_cols = ['POI_index',
                      'reviewer_name',
                      'raw_home_location',
                      'cleaned_home_location',
                      'number_of_contributions',
                      'helpful_votes',
                      'review_date',
                      'review_title',
                      'review_body',
                      'date_of_experience',
                      'trip_type',
                      'review_cralwed_time'
                     ]

poi_df = pd.DataFrame(columns=TripA_POI_cols)

### MVP ###
url_1 = 'https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-Gardens_by_the_Bay-Singapore.html'
url_2 = 'https://www.tripadvisor.com.sg/Attraction_Review-g294265-d1837767-Reviews-Marina_Bay_Sands_Skypark-Singapore.html'
driver.get(url_1)

# Attributes.
total_reviews_text = driver.find_element_by_xpath('//span[@class="reviewCount"]').text
ranking_text = driver.find_element_by_xpath('//span[@class="header_popularity popIndexValidation "]').text
rating_text = driver.find_element_by_xpath('//span[@class="overallRating"]').text
rating_proportion_elements = driver.find_elements_by_xpath('//span[@class="row_count row_cell"]')
address_text_1 = driver.find_element_by_xpath('//span[@class="street-address"]').text
address_text_2 = driver.find_element_by_xpath('//span[@class="extended-address"]').text
address_text_3 = driver.find_element_by_xpath('//span[@class="locality"]').text
address_text_4 = driver.find_element_by_xpath('//span[@class="country-name"]').text

try:
    about_more_button = driver.find_elements_by_xpath('//span[@class="attractions-attraction-detail-about-card-Description__readMore--2pd33"]')[0]
    about_more_button.click()
    sleep(0.5)
    about_text = driver.find_element_by_xpath('//div[@class="attractions-attraction-detail-about-card-Description__modalText--1oJCY"]').text
    about_more_close_button = driver.find_element_by_xpath('//div[@class="_2EFRp_bb"]')
    about_more_close_button.click()
    sleep(0.5)
    
except:
    about_text = driver.find_element_by_xpath('//div[@class="attractions-attraction-detail-about-card-AttractionDetailAboutCard__section--1_Efg"]').text

attributes_crawled_time = datetime.now()

print(total_reviews_text)
print(ranking_text)
print(rating_text)
for element in rating_proportion_elements:
    print(element.text)
print(about_text)
print(address_text_1)
print(address_text_2)
print(address_text_3)
print(address_text_4)
print(attributes_crawled_time)
print()

# Reviews
def parse_review_date(text):
    return text[(text.find("wrote a review ")+15):(text.find("\n"))]

def parse_review_date(text):
    return text[text.find('wrote a review ')+15:text.find('\n')]

def parse_location_contributions_votes(text):
    try:
        votes = int(re.search('(\d+) helpful votes?', text).group(1))
    except:
        votes = None
        
    try:
        contributions = int(re.search('(\d+) contributions?', text).group(1))
    except:
        contributions = None
        
    try:
        location = re.search('(.+?){} contributions?'.format(contributions), text).group(1)
    except:
        location = None
        
    return location, contributions, votes
    
def parse_review_rating(text):
    return int(text[-2:])//10

def parse_review_body(text):
    return text[:text.find('Read less')-1]

def parse_date_of_experience(text):
    substring = text[text.find('Date of experience: ')+20:]
    return substring[:substring.find('\n')]

def parse_trip_type(text):
    if text.find('Trip type: ') == -1:
        return None
    substring = text[text.find('Trip type: ')+11:]
    return substring[:substring.find('\n')]

for page in range(2):
    
    print('########## PAGE {} ##########\n'.format(page+1))
    
    read_more_button = driver.find_element_by_xpath('//span[@class="location-review-review-list-parts-ExpandableReview__cta--2mR2g"]')
    read_more_button.click()
    sleep(1)

    reviewer_url_elements = driver.find_elements_by_xpath('//a[@class="ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC"]')
    reviewer_details_elements = driver.find_elements_by_xpath('//div[@class="social-member-event-MemberEventOnObjectBlock__event_wrap--1YkeG"]')
    review_rating_elements = driver.find_elements_by_xpath('//div[@class="location-review-review-list-parts-RatingLine__bubbles--GcJvM"]/span')
    review_title_elements = driver.find_elements_by_xpath('//a[@class="location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT"]')
    review_body_elements = driver.find_elements_by_xpath('//div[@class="location-review-review-list-parts-ExpandableReview__containerStyles--1G0AE"]')
    date_of_experience_elements = driver.find_elements_by_xpath('//span[@class="location-review-review-list-parts-EventDate__event_date--1epHa"]')
    reviewer_name_elements = reviewer_url_elements
    review_crawled_time = datetime.now()
    
    for i in range(len(reviewer_url_elements)):
        print(reviewer_url_elements[i].get_attribute('href'))
        print(reviewer_name_elements[i].text)
        print(parse_review_date(reviewer_details_elements[i].text))
        print(parse_location_contributions_votes(reviewer_details_elements[i].text))
        print(parse_review_rating(review_rating_elements[i].get_attribute('class')))
        print(review_title_elements[i].text)
        print(parse_review_body(review_body_elements[i].text))
        print(parse_date_of_experience(review_body_elements[i].text))
        print(parse_trip_type(review_body_elements[i].text))
        print()

    next_button = driver.find_element_by_xpath('//a[@class="ui_button nav next primary "]')
    next_button.click()
    sleep(1)

58,471 Reviews
#3 of 2,760 things to do in Singapore
4.5
71%
23%
4%
1%
1%
An integral part of Singapore's "City in a Garden" vision, Gardens by the Bay spans a total of 101 hectares of prime land at the heart of Singapore's new downtown - Marina Bay. Comprising three waterfront gardens - Bay South, Bay East and Bay Central - Gardens by the Bay will be a showcase of horticulture and garden artistry that will bring the world of plants to Singapore and present Singapore to the World.
18 Marina Gardens Drive
Bayfront Plaza
Singapore 018953,
Singapore
2020-01-14 09:07:43.798657

########## PAGE 1 ##########

https://www.tripadvisor.com.sg/Profile/Tanya887
Tanya887
Today
('Brisbane, Australia', 11, 2)
5
A must whilst in singapore
A really lovely place to visit out of the hustle and bustle of the city. Beautiful gardens and an amazing setting. If in Singapore in December then the Xmas lights cannot be missed.
December 2019
None

https://www.tripadvisor.com.sg/Profile/Ull-123
Ull-123
Today
('G